In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# Show all output for a cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [3]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

In [4]:
import src.config as config
from src.inference import (
    get_feature_store
)

from datetime import datetime, timezone, timedelta

def get_current_utc_hour_ceiled():
    """Get current UTC time ceiled to the next hour"""
    current = datetime.now(timezone.utc)
    if current.minute > 0 or current.second > 0:
        # Ceil to next hour
        current = current + timedelta(hours=1)

    # Reset minutes, seconds, and microseconds
    current = current.replace(minute=0, second=0, microsecond=0)
    return current

# Usage
current_hour = get_current_utc_hour_ceiled()
print(f"Current UTC hour (ceiled): {current_hour}")

Current UTC hour (ceiled): 2025-05-11 13:00:00+00:00


In [5]:
def fetch_predictions():
    current_date = pd.Timestamp.now(tz='Etc/UTC').ceil('h')

    fs = get_feature_store()
    fg = fs.get_feature_group(
        name=config.FEATURE_GROUP_MODEL_PREDICTION,
        version=1
    )

    query = fg.select_all()
    query = query.filter(fg.pickup_hour == current_hour)

    return query.read()

In [6]:
fs = get_feature_store()
fg = fs.get_feature_group(
    name=config.FEATURE_GROUP_MODEL_PREDICTION,
    version=1
)

query = fg.select_all()

2025-05-11 08:54:10,377 INFO: Initializing external client
2025-05-11 08:54:10,378 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-05-11 08:54:11,114 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1231009


In [7]:
df= query.read()
df

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.48s) 


,start_station_id,predicted_demand,hour
0,6140.05,25.0,2025-05-11 13:00:00+00:00
1,5905.14,19.0,2025-05-11 13:00:00+00:00
2,5329.03,13.0,2025-05-11 13:00:00+00:00


In [8]:
from datetime import datetime, timezone, timedelta

def fetch_next_hour_predictions():
    # Get current UTC time and round up to next hour
    now = datetime.now(timezone.utc)
    next_hour = (now + timedelta(hours=1)).replace(minute=0, second=0, microsecond=0)

    # Extract components from the rounded-up time
    year = next_hour.year
    month = next_hour.month
    day = next_hour.day
    hour = next_hour.hour

    # Create date strings in YYYY-MM-DD format
    current_date = f"{year}-{month:02d}-{day:02d}"
    next_date = next_hour + timedelta(days=1)
    next_date_str = next_date.strftime("%Y-%m-%d")

    fs = get_feature_store()
    fg = fs.get_feature_group(
        name=config.FEATURE_GROUP_MODEL_PREDICTION,
        version=1
    )

    # # First get the full day's data from Hopsworks
    # df = fg.filter(
    #     (fg.pickup_hour >= current_date) &
    #     (fg.pickup_hour < next_date_str)
    # ).read()

    # Then filter for next hour in the DataFrame
    hour_str = f"{current_date} {hour:02d}:00"
    df_hour = df[df['hour'] == hour_str]

    print(f"Current UTC time: {now}")
    print(f"Next hour: {next_hour}")
    print(f"Querying for date range: {current_date} to {next_date_str}")
    print(f"Filtering for hour: {hour_str}")
    print(f"Found {len(df_hour)} records")
    return df_hour

# Example usage:
predictions = fetch_next_hour_predictions()

2025-05-11 08:54:13,783 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-05-11 08:54:13,791 INFO: Initializing external client
2025-05-11 08:54:13,791 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-05-11 08:54:14,369 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1231009
Current UTC time: 2025-05-11 12:54:13.783697+00:00
Next hour: 2025-05-11 13:00:00+00:00
Querying for date range: 2025-05-11 to 2025-05-12
Filtering for hour: 2025-05-11 13:00
Found 3 records


In [9]:
predictions

,start_station_id,predicted_demand,hour
0,6140.05,25.0,2025-05-11 13:00:00+00:00
1,5905.14,19.0,2025-05-11 13:00:00+00:00
2,5329.03,13.0,2025-05-11 13:00:00+00:00


In [10]:
import pandas as pd
current_date = pd.Timestamp.now(tz='Etc/UTC').ceil('h')

In [11]:
dt = current_date.ceil('h')

In [12]:
fs = get_feature_store()  
fg = fs.get_feature_group(  
    name=config.FEATURE_GROUP_MODEL_PREDICTION,  
    version=1  
)  
query = fg.select_all() 
# query = query.filter(fg.pickup_hour >= "2025-02-13 04:00")
query = query.filter(fg.hour > "2025-05-10 05:00:00")  
results = query.read()

2025-05-11 08:54:15,446 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-05-11 08:54:15,450 INFO: Initializing external client
2025-05-11 08:54:15,451 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-05-11 08:54:15,938 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1231009
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.40s) 


In [13]:
results

,start_station_id,predicted_demand,hour
0,6140.05,25.0,2025-05-11 13:00:00+00:00
1,5905.14,19.0,2025-05-11 13:00:00+00:00
2,5329.03,13.0,2025-05-11 13:00:00+00:00


In [14]:
def fetch_next_hour_predictions():
    # Get current UTC time and round up to next hour
    now = datetime.now(timezone.utc)
    next_hour = (now + timedelta(hours=1)).replace(minute=0, second=0, microsecond=0)

    fs = get_feature_store()
    fg = fs.get_feature_group(name=config.FEATURE_GROUP_MODEL_PREDICTION, version=1)

    # Then filter for next hour in the DataFrame
    df_hour = df[df["hour"] == next_hour]

    print(f"Current UTC time: {now}")
    print(f"Next hour: {next_hour}")
    print(f"Found {len(df_hour)} records")
    return df_hour

In [15]:
predictions = fetch_next_hour_predictions()

2025-05-11 08:54:17,784 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-05-11 08:54:17,791 INFO: Initializing external client
2025-05-11 08:54:17,792 INFO: Base URL: https://c.app.hopsworks.ai:443


2025-05-11 08:54:18,332 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1231009
Current UTC time: 2025-05-11 12:54:17.784944+00:00
Next hour: 2025-05-11 13:00:00+00:00
Found 3 records


In [16]:
fs = get_feature_store()
fg = fs.get_feature_group(name=config.FEATURE_GROUP_MODEL_PREDICTION, version=1)

2025-05-11 08:54:19,065 INFO: Closing external client and cleaning up certificates.


Connection closed.
2025-05-11 08:54:19,071 INFO: Initializing external client
2025-05-11 08:54:19,072 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-05-11 08:54:19,613 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1231009


In [17]:
df = fg.read()

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.64s) 


In [18]:
df["hour"].max()

Timestamp('2025-05-11 13:00:00+0000', tz='Etc/UTC')

In [19]:
now = datetime.now(timezone.utc)
next_hour = (now + timedelta(hours=1)).replace(minute=0, second=0, microsecond=0)
print(next_hour)

2025-05-11 13:00:00+00:00


In [20]:
df[df["hour"] == next_hour]

,start_station_id,predicted_demand,hour
0,6140.05,25.0,2025-05-11 13:00:00+00:00
1,5905.14,19.0,2025-05-11 13:00:00+00:00
2,5329.03,13.0,2025-05-11 13:00:00+00:00
